In [1]:
from google.colab import drive
drive.mount('/content/drive')
main_directory = '/content/drive/My Drive/Thesis/Thesis Codes 4_2/Dataset/train.csv'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import pandas as pd
data = pd.read_csv(main_directory,encoding = 'ISO-8859-1')
data.head()

,label,id,date,query,user,tweet
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [6]:
data.shape

(1048575, 6)

In [5]:
data.label.unique()

array([0, 4])

In [3]:
from __future__ import division, print_function
from gensim import models
from keras.callbacks import ModelCheckpoint
from keras.layers import Dense, Dropout, Reshape, Flatten, concatenate, Input, Conv1D, GlobalMaxPooling1D, Embedding
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import os
import collections
import re
import string

Using TensorFlow backend.


In [0]:
pos = []
neg = []
for l in data.label:
  if l == 0:
     pos.append(0)
     neg.append(1)
  elif l == 4:
     pos.append(1)
     neg.append(0)

In [0]:
data['Pos']= pos
data['Neg']= neg

In [9]:
data.head()

,label,id,date,query,user,tweet,Pos,Neg
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,0,1
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,0,1
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,0,1
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",0,1
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew,0,1


In [0]:

def remove_punct(text):
   text_nopunct = ''
   text_nopunct = re.sub('['+string.punctuation+']', '',text)
   return text_nopunct
   
data['Text_Clean'] =data['tweet'].apply(lambda x: remove_punct(x))

In [11]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
from nltk import word_tokenize, WordNetLemmatizer
tokens = [word_tokenize(tweet) for tweet in data.Text_Clean]

In [0]:
def lowerStemmer(tokens): 
    return [w.lower() for w in tokens]    
    
lower_tokens = [lowerStemmer(token) for token in tokens]

In [14]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
from nltk.corpus import stopwords
stoplist = stopwords.words('english')

In [0]:
def removeStopWords(tokens): 
    return [word for word in tokens if word not in stoplist]

In [0]:
filtered_words = [removeStopWords(sen) for sen in lower_tokens]

In [0]:
result = [' '.join(sen) for sen in filtered_words]

In [0]:
data['Text_Final'] = result

In [0]:
data['tokens'] = filtered_words

In [0]:
data = data[['Text_Final', 'tokens', 'label', 'Pos', 'Neg']]

In [23]:
data[:4]

,Text_Final,tokens,label,Pos,Neg
0,upset cant update facebook texting might cry r...,"[upset, cant, update, facebook, texting, might...",0,0,1
1,kenichan dived many times ball managed save 50...,"[kenichan, dived, many, times, ball, managed, ...",0,0,1
2,whole body feels itchy like fire,"[whole, body, feels, itchy, like, fire]",0,0,1
3,nationwideclass behaving im mad cant see,"[nationwideclass, behaving, im, mad, cant, see]",0,0,1


In [0]:
data_train, data_test = train_test_split(data, test_size=0.10, random_state=42)

In [25]:
all_training_words = [word for tokens in data_train["tokens"] for word in tokens]
training_sentence_lengths = [len(tokens) for tokens in data_train["tokens"]]
TRAINING_VOCAB = sorted(list(set(all_training_words)))
print("%s words total, with a vocabulary size of %s" % (len(all_training_words), len(TRAINING_VOCAB)))
print("Max sentence length is %s" % max(training_sentence_lengths))

7468188 words total, with a vocabulary size of 550175
Max sentence length is 56


In [26]:
all_test_words = [word for tokens in data_test["tokens"] for word in tokens]
test_sentence_lengths = [len(tokens) for tokens in data_test["tokens"]]
TEST_VOCAB = sorted(list(set(all_test_words)))
print("%s words total, with a vocabulary size of %s" % (len(all_test_words), len(TEST_VOCAB)))
print("Max sentence length is %s" % max(test_sentence_lengths))

830537 words total, with a vocabulary size of 109345
Max sentence length is 47


In [28]:
word2vec_path = 'https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz'
word2vec = models.KeyedVectors.load_word2vec_format(word2vec_path, binary=True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
def get_average_word2vec(tokens_list, vector, generate_missing=False, k=300):
    if len(tokens_list)<1:
        return np.zeros(k)
    if generate_missing:
        vectorized = [vector[word] if word in vector else np.random.rand(k) for word in tokens_list]
    else:
        vectorized = [vector[word] if word in vector else np.zeros(k) for word in tokens_list]
    length = len(vectorized)
    summed = np.sum(vectorized, axis=0)
    averaged = np.divide(summed, length)
    return averaged

def get_word2vec_embeddings(vectors, clean_comments, generate_missing=False):
    embeddings = clean_comments['tokens'].apply(lambda x: get_average_word2vec(x, vectors, 
                                                                                generate_missing=generate_missing))
    return list(embeddings)

In [0]:
training_embeddings = get_word2vec_embeddings(word2vec, data_train, generate_missing=True)

In [0]:
MAX_SEQUENCE_LENGTH = 50
EMBEDDING_DIM = 300

In [32]:
tokenizer = Tokenizer(num_words=len(TRAINING_VOCAB), lower=True, char_level=False)
tokenizer.fit_on_texts(data_train["Text_Final"].tolist())
training_sequences = tokenizer.texts_to_sequences(data_train["Text_Final"].tolist())

train_word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(train_word_index))

Found 550100 unique tokens.


In [0]:
train_cnn_data = pad_sequences(training_sequences, maxlen=MAX_SEQUENCE_LENGTH)

In [34]:
train_embedding_weights = np.zeros((len(train_word_index)+1, EMBEDDING_DIM))
for word,index in train_word_index.items():
    train_embedding_weights[index,:] = word2vec[word] if word in word2vec else np.random.rand(EMBEDDING_DIM)
print(train_embedding_weights.shape)

(550101, 300)


In [0]:
test_sequences = tokenizer.texts_to_sequences(data_test["Text_Final"].tolist())
test_cnn_data = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)

In [0]:
def ConvNet(embeddings, max_sequence_length, num_words, embedding_dim, labels_index):
    
    embedding_layer = Embedding(num_words,
                            embedding_dim,
                            weights=[embeddings],
                            input_length=max_sequence_length,
                            trainable=False)
    
    sequence_input = Input(shape=(max_sequence_length,), dtype='int32')
    embedded_sequences = embedding_layer(sequence_input)

    convs = []
    filter_sizes = [2,3,4,5,6]

    for filter_size in filter_sizes:
        l_conv = Conv1D(filters=200, kernel_size=filter_size, activation='relu')(embedded_sequences)
        l_pool = GlobalMaxPooling1D()(l_conv)
        convs.append(l_pool)


    l_merge = concatenate(convs, axis=1)

    x = Dropout(0.1)(l_merge)  
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.2)(x)
    preds = Dense(labels_index, activation='sigmoid')(x)

    model = Model(sequence_input, preds)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['acc'])
    model.summary()
    return model

In [0]:
label_names = ['Pos', 'Neg']

In [0]:
y_train = data_train[label_names].values

In [0]:
x_train = train_cnn_data
y_tr = y_train

In [40]:
model = ConvNet(train_embedding_weights, MAX_SEQUENCE_LENGTH, len(train_word_index)+1, EMBEDDING_DIM, 
                len(list(label_names)))











Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 50)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 50, 300)      165030300   input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 49, 200)      120200      embedding_1[0][0]                
_____________________________________________________________________

In [0]:
num_epochs = 3
batch_size = 34

In [42]:
hist = model.fit(x_train, y_tr, epochs=num_epochs, validation_split=0.1, shuffle=True, batch_size=batch_size)



Train on 849345 samples, validate on 94372 samples
Epoch 1/3
203116/849345 [======>.......................] - ETA: 31:35 - loss: 0.4188 - acc: 0.8105WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:1033: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.


Train on 849345 samples, validate on 94372 samples
Epoch 1/3
849345/849345 [==============================] - 2576s 3ms/step - loss: 0.3938 - acc: 0.8239 - val_loss: 0.3760 - val_acc: 0.8354
Epoch 2/3
849345/849345 [==============================] - 2566s 3ms/step - loss: 0.3653 - acc: 0.8383 - val_loss: 0.3629 - val_acc: 0.8385
Epoch 3/3
151946/849345 [====>.........................] - ETA: 33:56 - loss: 0.3434 - acc: 0.8475Buffered data was truncated after reaching the output size limit.Buffered data was truncated after reaching the output size limit.

In [43]:
predictions = model.predict(test_cnn_data, batch_size=1024, verbose=1)

104858/104858 [==============================] - 102s 968us/step


In [0]:
labels = [4, 0]

In [0]:
prediction_labels=[]
for p in predictions:
    prediction_labels.append(labels[np.argmax(p)])

In [47]:
sum(data_test.label==prediction_labels)/len(prediction_labels)

0.8412615155734422

In [49]:
data_test.label.value_counts()

0    80131
4    24727
Name: label, dtype: int64